In [ ]:
#step1 - audio file to test conversion

!pip install speechRecognition

In [ ]:
import speech_recognition as spr

In [ ]:
recognizer=spr.Recognizer()

In [ ]:
#kindly upload the desired audio file that has to be converted

proverb1=spr.AudioFile('/content/recording2023-10-05-23-59-23.wav') #the input audio file

In [ ]:
with proverb1 as source:
  proverb1 = recognizer.record(source,duration=120,offset=11)

In [ ]:
# Add the language spoken in the audio in text --> language
text = recognizer.recognize_google(proverb1, language="ta-IN")
print(text)

In [ ]:
with open ('writeme.txt', 'w') as file:
    file.write(text)

In [ ]:
#step2 --> translation
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator

file_translate = Translator()

In [ ]:
result = file_translate.translate(text)
print(result.text)

In [ ]:
with open ('speech.txt', 'w') as file:
    file.write(result.text)

In [ ]:
#step3 --> text to audio conversion

!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

In [ ]:
# Here we split the text into smaller forms so that the audio can be easily generated.
with open('/content/speech.txt', 'r') as f:
   lines = f.read().split('.')

# **** NOTE *****

# If you have GPU inbuilt in your system you can skip the above code and below cells and run the following code given below

# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.

CUSTOM_VOICE_NAME = "speech1"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

from tortoise.utils.text import split_and_recombine_text
from time import time
import os

outpath = "results/longform/"

textfile_path = "../speech.txt"

# Process text
with open(textfile_path, 'r', encoding='utf-8') as f:
    text = ' '.join([l for l in f.readlines()])
    if '|' in text:
        print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
              "your intent, please remove all '|' characters from the input.")
        texts = text.split('|')
    else:
        texts = split_and_recombine_text(text)

seed = int(time())

voice_outpath = os.path.join(outpath, CUSTOM_VOICE_NAME)
os.makedirs(voice_outpath, exist_ok=True)

voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

all_parts = []
for j, text in enumerate(texts):
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                              preset="standard", k=1, use_deterministic_seed=seed)
    gen = gen.squeeze(0).cpu()
    torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), gen, 24000)
    all_parts.append(gen)

full_audio = torch.cat(all_parts, dim=-1)
torchaudio.save(os.path.join(voice_outpath, 'combined.wav'), full_audio, 24000)
IPython.display.Audio(os.path.join(voice_outpath, 'combined.wav'))




In [ ]:
n=len(lines)
di={}
for i in range(1,n):
  di[i]=lines[i]

In [ ]:
import os
from google.colab import files
from torchaudio.utils import download_asset

#

# This is the text that will be spoken.
#"I told him that he went to north according to the Guru's wish and he went to China. As soon as heads there, a disciple of the Buddha or someone immersed in his path would come this way. The news spread that MS had this desire all his life."

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py

preset = "high_quality" # I have chosen "high_quality"

# upload at least 2 sample audios. They must be a WAV file, 6-10 seconds long.

for k in di.keys():
  CUSTOM_VOICE_NAME="audio{fname}".format(fname = k)
  text=di[k]
  custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
  os.makedirs(custom_voice_folder)
  for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
      f.write(file_data)
  voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
  gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
  torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
  torchaudio.save('/content/'+f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
  IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')


In [ ]:
# Once the previous code has run, upload all your saved recordings to combine them into a single folder.

from pydub import AudioSegment
sound1 = AudioSegment.from_file("/path/to/sound.wav", format="wav")
sound2 = AudioSegment.from_file("/path/to/another_sound.wav", format="wav")
sound3 = AudioSegment.from_file("/path/to/another_sound.wav", format="wav")
sound4 = AudioSegment.from_file("/path/to/another_sound.wav", format="wav")
sound5 = AudioSegment.from_file("/path/to/another_sound.wav", format="wav")

# sound1 6 dB louder
louder = sound1 + 6


# sound1, with sound2 appended (use louder instead of sound1 to append the louder version)
combined = sound1 + sound2 + sound3 + sound4 + sound5 + etc...

# simple export
file_handle = combined.export("/path/to/output.mp3", format="mp3")